# Lag STAC fra GeoTiff

### Input: 
Mappe med .tif og .tfw -filer. Filer kan lastes ned fra Geonorge her: [(N50 Raster (UTM32) - Rutevis)](https://kartkatalog.geonorge.no/metadata/n50-raster-utm32-rutevis/bbe517ba-3b3a-4c96-a78e-104f6c4804d1?search=n50)

### Output:
**/stac_catalog** som inneholder *catalog.json* (oversikt) og egen mappe for hvert innleste kartblad.

## Import av bibliotek 

In [ ]:
import os
import rasterio
import utils
import shutil
import subprocess
from pathlib import Path
from pystac import Catalog, Item, Asset
from datetime import datetime
from shapely.geometry import box, mapping

: 

## Mappestruktur
Her er det tenkt at koden kjøres i en mappe som har underkatalogene "Input" (inneholder bilder) og "Output" (mål for STAC-katalog)

In [ ]:
# Definer input/output mapper
inputFolder = utils.get_workdir() / "Input"
outputFolder = utils.get_workdir() / "Output"

# Opprett COG-mappe hvis ikke den eksisterer
COGFolder = utils.get_workdir() / "COG"
if COGFolder.exists():
    for item in COGFolder.iterdir():
        if item.is_file():
            item.unlink()
        elif item.is_dir():
            shutil.rmtree(item)
else:
    COGFolder.mkdir(parents=True, exist_ok=True)

catalog = Catalog(id="N50-test-catalog", description="STAC Catalog for N50 maps")

## UDF: Konverter GeoTiff til COG

In [ ]:
def convert_folder_to_cog(input_folder, COGFolder):
    input_folder = Path(input_folder)
    COGFolder = Path(COGFolder)
    COGFolder.mkdir(parents=True, exist_ok=True)

    for tif_file in input_folder.glob("*.tif"):
        output_file = COGFolder / tif_file.name  # Same name
        print(f"Konverterer {tif_file.name} til COG...")
        
        cmd = [
            "gdal_translate", str(tif_file), str(output_file),
            "-of", "COG",
            "-co", "COMPRESS=DEFLATE",
            "-co", "TILING_SCHEME=GoogleMapsCompatible"
        ]

        try:
            subprocess.run(cmd, check=True)
        except subprocess.CalledProcessError as e:
            print(f"Konvertering misslykkes for {tif_file.name}: {e}")

    print("Alle filer konvertert.")

## Les og legg filene til i STAC-katalogen

In [ ]:
convert_folder_to_cog(inputFolder, COGFolder)

for filename in os.listdir(COGFolder):
    if filename.endswith(".cog"): # Velg kun TIFF-filer
        filepath = COGFolder / filename

        # Les bbox og EPSG-kode fra TIFF-filen
        with rasterio.open(filepath) as src:
            bounds = src.bounds
            bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
            geometry = mapping(box(*bbox))
            crs = src.crs.to_string()

        item_id = os.path.splitext(filename)[0]
        item = Item(
            id=item_id,
            geometry=geometry,
            bbox=bbox,
            datetime=datetime.now(),
            properties={"proj:epsg": int(crs.split(":")[-1])},
        )

        # Output-katalog
        item_output_dir = outputFolder / "stac_catalog" / item_id
        item_output_dir.mkdir(parents=True, exist_ok=True)
        item_path = item_output_dir / f"{item_id}.json"

        # Relativ sti til bildet
        rel_image_href = os.path.relpath(filepath, start=item_path.parent)
        item.add_asset("image", Asset(href=rel_image_href, media_type="image/tiff"))

        # Hvis tfw-hjelpefiler finnes legges de til som "georef"
        twf_path = inputFolder / f"{item_id}.tfw"
        if twf_path.exists():
            rel_tfw_href = os.path.relpath(twf_path, start=item_path.parent)
            item.add_asset(
                "georef",
                Asset(
                    href=rel_tfw_href,
                    media_type="text/plain",
                    roles=["auxiliary", "georeferencing"],
                ),
            )

        catalog.add_item(item)

# Lagre katalogen
catalog.normalize_and_save(
    root_href=str(outputFolder / "stac_catalog"),
    catalog_type="SELF_CONTAINED"
)